In [3]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn

- σ(x)容易导致梯度消失
- relu(x)部分改善梯度消失
    - 会导致稀疏表示sparse representation
    - dying relu/ dead neuron
- torch.clamp(x, min, max)
    - relu/clamp的gradient都是0-1组成的mask tensor

- $\sigma(x)(1-\sigma(x))$  **反向梯度链式法则计算，连乘导致越来越小**

In [4]:
x = torch.normal(0,1,(3,5))
w = nn.Linear(5,4)
w(x)

tensor([[-0.6969, -0.2325,  1.2014, -0.9012],
        [-0.5177, -0.3649,  0.4428,  0.1119],
        [ 0.2828,  1.0449,  0.8982, -0.7627]], grad_fn=<AddmmBackward0>)

In [5]:
torch.relu(w(x))

tensor([[0.0000, 0.0000, 1.2014, 0.0000],
        [0.0000, 0.0000, 0.4428, 0.1119],
        [0.2828, 1.0449, 0.8982, 0.0000]], grad_fn=<ReluBackward0>)

异常输入——>大的relu输出——>反向传播大的梯度——>反向更新后，输出变成负数——>gradient变为零

torch.clamp(x, min, max)

$$y^{'}=
\left \{ \begin{matrix}
 &0 ,&x<xmin \\ 
 &1 ,&xmin<x<xmax\\
 &0 ,&x>xmin
\end{matrix} \right.
$$


In [56]:
x = torch.tensor([[1.,2.,3.],
          ],requires_grad=True)
print(x)
w = torch.tensor([[1.],[-2.],[3.]],requires_grad=True)
print(w)
z = torch.matmul(x, w)
print(z)
y = torch.clamp(z,-8,8)
print(y)
y.sum().backward()

tensor([[1., 2., 3.]], requires_grad=True)
tensor([[ 1.],
        [-2.],
        [ 3.]], requires_grad=True)
tensor([[6.]], grad_fn=<MmBackward0>)
tensor([[6.]], grad_fn=<ClampBackward1>)


In [57]:
w.grad

tensor([[1.],
        [2.],
        [3.]])

In [58]:
x.grad

tensor([[ 1., -2.,  3.]])

In [51]:
x = torch.tensor([[1.,2.,3.],
               [4.,5.,6.],
                [5.,3.,2],
                 [2,-1,6] ],requires_grad=True)
print(x)
w = torch.tensor([[1.],[-2.],[3.]],requires_grad=True)
print(w)
z = torch.matmul(x, w)
print(z)
y = torch.clamp(z,-8,8)
print(y)
y.sum().backward()

tensor([[ 1.,  2.,  3.],
        [ 4.,  5.,  6.],
        [ 5.,  3.,  2.],
        [ 2., -1.,  6.]], requires_grad=True)
tensor([[ 1.],
        [-2.],
        [ 3.]], requires_grad=True)
tensor([[ 6.],
        [12.],
        [ 5.],
        [22.]], grad_fn=<MmBackward0>)
tensor([[6.],
        [8.],
        [5.],
        [8.]], grad_fn=<ClampBackward1>)


In [52]:
w.grad, x.grad

(tensor([[6.],
         [5.],
         [5.]]),
 tensor([[ 1., -2.,  3.],
         [ 0., -0.,  0.],
         [ 1., -2.,  3.],
         [ 0., -0.,  0.]]))